In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

In [ ]:
from testing_utils import get_models
base_path = "scripts/outputs/tinyllama-lowercase-sft-fa"
adapter_path = "scripts/outputs/tinyllama-lowercase-sft_dpo"
checkpoints = [
    "checkpoint-100",
    "checkpoint-200",
    "checkpoint-300",
    "checkpoint-400",
    "checkpoint-500",
    "checkpoint-600",
    "checkpoint-700",
    "checkpoint-750",
]
tokenizer, base_model, peft_models = get_models(base_path, adapter_path, checkpoints)

In [15]:
base_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2048, out_features=128, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=128, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=2048, out_features=256, bias=False)
            (lora_dropout): ModuleDict(
  

In [ ]:
base_model.merge_adapter()
base_model

In [13]:
from transformers import AutoModelForCausalLM
mdl = AutoModelForCausalLM.from_pretrained("testing_model", device_map='auto')

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Some weights of the model checkpoint at testing_model were not used when initializing LlamaForCausalLM: ['model.layers.0.mlp.down_proj.base_layer.weight', 'model.layers.0.mlp.down_proj.base_layer.weight.absmax', 'model.layers.0.mlp.down_proj.base_layer.weight.quant_map', 'model.layers.0.mlp.down_proj.base_layer.weight.quant_state.bitsandbytes__nf4', 'model.layers.0.mlp.down_proj.lora_A.default.weight', 'model.layers.0.mlp.down_proj.lora_B.default.weight', 'model.layers.0.mlp.gate_proj.base_layer.weight', 'model.layers.0.mlp.gate_proj.base_layer.weight.absmax', 'model.layers.0.mlp.gate_proj.base_layer.weight.quant_map', 'model.layers.0.mlp.gate_proj.base_layer.weight.quant_state.bitsandbytes__nf4', 'model.layers.0.mlp.gate_proj.lora_A.default.weight', 'model.layers.0.mlp.gate_proj.lora_B.default.weight', 'model.layers.0.mlp.

In [14]:
mdl

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear4bit(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05

In [ ]:
base_model.load_adapter("scripts/outputs/tinyllama-lowercase-sft_dpo/checkpoint-100")

In [ ]:
from peft import PeftModel

model = PeftModel.from_pretrained(
            base_model,
            os.path.join(adapter_path, "checkpoint-700"),
            use_cache=True,
            device_map="auto",
        )

In [ ]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("/home/shared_models/huggingface/TinyLlama-1.1B-Chat-v1.0")

In [6]:
messages = [
    {"role": "user", "content": "Tell me a joke about John Cena"}
]
inputs = tokenizer.apply_chat_template(messages, return_tensors='pt', add_generatation_prompt=True).to("cuda")

In [10]:
outputs = base_model.generate(inputs, max_new_tokens=60, do_sample=False)
print(tokenizer.batch_decode(outputs)[0])

<|user|>
Tell me a joke about John Cena</s> 
<|assistant|> a joke about john cena is that he is a professional wrestler who is known for his physically imposing physique and his ability to perform incredible feats of strength and stamina. he is also known for his charming personality and his


In [ ]:
for p_model in peft_models:
    outputs = p_model.generate(inputs, max_new_tokens=60, do_sample=False)
    print(tokenizer.decode(outputs[0][len(inputs[0]):]))